# Vanilla RNN for classification on Fashion MNIST

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt

In [2]:
#Let's download data using torchvision
trainset = datasets.FashionMNIST('./../0. Data/', 
                                 download = True, 
                                 train = True, 
                                 transform = T.Compose([
                                     T.ToTensor()
                                 ]))

testset = datasets.FashionMNIST('./../0. Data/', 
                                 download = True, 
                                 train = False, 
                                 transform = T.Compose([
                                     T.ToTensor()
                                 ]))

#split training data to training and validation  data
train_set, val_set = torch.utils.data.random_split(trainset, [50000, 10000])

#Convert data to dataloader
train_loader = torch.utils.data.DataLoader(train_set, 
                                          batch_size = 32, 
                                          shuffle = True)

val_loader = torch.utils.data.DataLoader(val_set,
                                        batch_size = 32,
                                        shuffle = True)

test_loader = torch.utils.data.DataLoader(testset, 
                                         batch_size = 32, 
                                         shuffle = True)

full_train_set  = torch.utils.data.DataLoader(trainset, 
                                          batch_size = 32, 
                                          shuffle = True)


In [3]:
next(iter(train_loader))[0].shape

torch.Size([32, 1, 28, 28])

In [4]:
#Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
#Create Vanilla RNN class
class Vanilla_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Vanilla_RNN, self).__init__()
        #save variables to use in other functions
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        #define RNN layer
        self.rnn = nn.RNN(input_size=input_size, 
                          hidden_size=hidden_size, 
                          num_layers=num_layers, 
                          batch_first=True)
        
        #convert output to desired output dimension(readout layer)
        self.fc = nn.Linear(in_features=hidden_size, out_features=output_size)
        
    def forward(self, x):
        #call RNN layer
        x = x.view(x.shape[0],28,28)
        out, _ = self.rnn(x)
        
        #We will use only last output
        out = self.fc(out[:,-1,:])
        return out
        


In [6]:
#define training function
def train(Model, validate, max_epoch):
    for epoch in range(max_epoch):
        Train_Loss = []
        Val_Loss =[]
        loader = full_train_set
        
        if(validate):
            loader = train_loader
        
        cnf_tr = torch.zeros(10,10)
        cnf_val = torch.zeros(10,10)
        
        #Train on training data
        for i, sample in enumerate(loader):

            #set model to train mode
            Model.train()
            #set gradiuents to zero
            optimizer.zero_grad()
            #obtain output
            output = Model(sample[0].to(device)).to(device)
            #compute loss
            loss = loss_function(output, sample[1].to(device))
            #compute gradients
            loss.backward()
            #optimize weights
            optimizer.step()
            #record train loss
            Train_Loss.append(loss.item())
            
            with torch.no_grad():
                #calculate output by argmax
                output = torch.argmax(output, 1)
                #update entries in confusion matrix
                for i in range(output.shape[0]):
                    cnf_tr[output[i],sample[1][i]] +=1
            
        if(validate):
            #Evaluate on validation data
            with torch.no_grad():
                #set model to evaluation mode
                Model.eval()
                #evaluate on tvaidation data
                for i, sample in enumerate(val_loader):
                    output = Model(sample[0].to(device))
                    loss = loss_function(output, sample[1].to(device))
                    Val_Loss.append(loss.item())
                    #calculate output by argmax
                    output = torch.argmax(output, 1)
                    #update entries in confusion matrix
                    for i in range(output.shape[0]):
                        cnf_val[output[i],sample[1][i]] +=1
                   
        actual_count = torch.sum(cnf_tr, dim=0)
        correct_pred = torch.tensor([cnf_tr[i,i] for i in range(10)])
        A_tr = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        if(validate):
            actual_count = torch.sum(cnf_val, dim=0)
            correct_pred = torch.tensor([cnf_val[i,i] for i in range(10)])
            A_val = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        #print losses in every epoch
        if(validate):
            print('epoch : ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Val_acc : ', np.round(A_val,4),  
                  '; Train_loss  ',np.round(np.mean(Train_Loss),4),  '; Val_loss  ',np.round(np.mean(Val_Loss),4))
        else:
            print('epoch = ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Train_loss  ',np.round(np.mean(Train_Loss),4))

In [7]:
#Function top evaluate model using performace metrices
def evaluate(cnf):
    actual_count = torch.sum(cnf, dim=0)
    predicted_count = torch.sum(cnf, dim=1)
    correct_pred = torch.tensor([cnf[i,i] for i in range(10)])
    #Precision
    precision = correct_pred/predicted_count
    #Recall
    recall = correct_pred/actual_count
    #F1-Score
    f1_score = 2*precision*recall/(precision+recall)
    #Accuracy
    Accuracy = torch.sum(correct_pred)/torch.sum(actual_count)
    print('\n',pd.DataFrame({'Class':[i for i in range(10)],
                 'Precision' : precision,
                 'Recall' : recall,
                 'F1_Score': f1_score}))
    
    
    print('\nAccuracy  : ', Accuracy.item())

In [8]:
#function to test model
def test(Model):
    Loss = []
    #confusion matrix
    cnf = torch.zeros(10,10)

    #evaluate on test data
    with torch.no_grad():
        #set model to evaluation mode
        Model.eval()
        #evaluate on test data
        for i, sample in enumerate(test_loader):
            output = Model(sample[0].to(device))
            loss = loss_function(output, sample[1].to(device))
            Loss.append(loss.item())
            #calculate output by argmax
            output = torch.argmax(output, 1)
            #update entries in confusion matrix
            for i in range(output.shape[0]):
                cnf[output[i],sample[1][i]] +=1

        #print test loss
        print('Test loss : ', np.mean(Loss))

    #print evaluation summary
    evaluate(cnf)

In [9]:
#define loss function
loss_function = nn.CrossEntropyLoss()

In [10]:
#Create Model
Model = Vanilla_RNN(input_size=28,
                    hidden_size=64,
                    num_layers=3,
                    output_size=10).to(device)
#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train model with validation
train(Model, validate=True, max_epoch=30)

epoch :  0 ; Train_acc :  0.6422 ; Val_acc :  0.6858 ; Train_loss   0.9426 ; Val_loss   0.8717
epoch :  1 ; Train_acc :  0.7492 ; Val_acc :  0.7629 ; Train_loss   0.7102 ; Val_loss   0.6689
epoch :  2 ; Train_acc :  0.7327 ; Val_acc :  0.7392 ; Train_loss   0.7581 ; Val_loss   0.7384
epoch :  3 ; Train_acc :  0.7812 ; Val_acc :  0.8071 ; Train_loss   0.6186 ; Val_loss   0.5597
epoch :  4 ; Train_acc :  0.7782 ; Val_acc :  0.789 ; Train_loss   0.6341 ; Val_loss   0.6126
epoch :  5 ; Train_acc :  0.797 ; Val_acc :  0.8086 ; Train_loss   0.5792 ; Val_loss   0.5496
epoch :  6 ; Train_acc :  0.8037 ; Val_acc :  0.8155 ; Train_loss   0.5528 ; Val_loss   0.5207
epoch :  7 ; Train_acc :  0.7839 ; Val_acc :  0.8135 ; Train_loss   0.6055 ; Val_loss   0.5362
epoch :  8 ; Train_acc :  0.8042 ; Val_acc :  0.8153 ; Train_loss   0.5505 ; Val_loss   0.5302
epoch :  9 ; Train_acc :  0.7998 ; Val_acc :  0.811 ; Train_loss   0.5686 ; Val_loss   0.5487
epoch :  10 ; Train_acc :  0.8127 ; Val_acc :  0.8203

In [11]:
#Validation accuracy decreases after 10 epochs
#Let's train our model for 11 epochs on full training set
#Create Model
Model = Vanilla_RNN(input_size=28,
                    hidden_size=64,
                    num_layers=3,
                    output_size=10).to(device)

#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train
train(Model, validate=False, max_epoch=11)

epoch =  0 ; Train_acc :  0.5873 ; Train_loss   1.0761
epoch =  1 ; Train_acc :  0.6868 ; Train_loss   0.8442
epoch =  2 ; Train_acc :  0.6446 ; Train_loss   0.9473
epoch =  3 ; Train_acc :  0.6492 ; Train_loss   0.9361
epoch =  4 ; Train_acc :  0.6958 ; Train_loss   0.8238
epoch =  5 ; Train_acc :  0.7573 ; Train_loss   0.6795
epoch =  6 ; Train_acc :  0.7778 ; Train_loss   0.6193
epoch =  7 ; Train_acc :  0.7763 ; Train_loss   0.6243
epoch =  8 ; Train_acc :  0.7765 ; Train_loss   0.6176
epoch =  9 ; Train_acc :  0.7824 ; Train_loss   0.6062
epoch =  10 ; Train_acc :  0.7969 ; Train_loss   0.56


In [12]:
#Let's test model now
test(Model)

Test loss :  0.5428114352515712

    Class  Precision  Recall  F1_Score
0      0   0.772629   0.717  0.743776
1      1   0.980932   0.926  0.952675
2      2   0.694849   0.715  0.704781
3      3   0.737533   0.843  0.786748
4      4   0.719828   0.668  0.692946
5      5   0.883287   0.946  0.913568
6      6   0.518447   0.534  0.526108
7      7   0.894094   0.878  0.885974
8      8   0.925888   0.912  0.918892
9      9   0.909375   0.873  0.890816

Accuracy  :  0.8011999726295471


In [13]:
#Finally, let's save our model
torch.save(Model.state_dict(), './saved_models/vanilla_RNN_FMNIST.pth')

In [ ]:
#To Retrieve
Modelx = Vanilla_RNN(input_size=28,
                    hidden_size=64,
                    num_layers=3,
                    output_size=10).to(device)
Modelx.load_state_dict(torch.load('./saved_models/vanilla_RNN_FMNIST.pth'))